In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [20]:
train=pd.read_csv('../data/train_상장_1대1.csv', encoding='euc-kr')
test=pd.read_csv('../data/test_상장_1대1.csv',encoding='euc-kr')

In [21]:
X_train_int=train[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름',
       '자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률']]

X_test_int=test[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름',
       '자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률']]

In [22]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [23]:
X_train_sc = X_train
X_test_sc = X_test

In [24]:
X_train_sum =X_train_sc
X_test_sum =X_test_sc

In [25]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import numpy as np

def perform_lgbm_grid_search(X_train, y_train, k_fold=5):
    # Stratified k-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # LGBM 하이퍼파라미터 후보 리스트 설정
    param_grid = {
        'max_depth': [3, 5, 8],
        'n_estimators': [50, 100, 150],
        'learning_rate': [50, 100, 150],
        'objective': ['binary'],
        'random_state': [0]
    }

    # LGBM 모델 초기화
    model = lgb.LGBMClassifier()

    # 그리드 서치 설정
    grid_search = GridSearchCV(model, param_grid, scoring='f1', cv=cv, verbose=1, n_jobs=-1)

    # 모델 학습 및 튜닝
    grid_search.fit(X_train, y_train)

    # 최적 하이퍼파라미터 출력
    print("Best Hyperparameters:", grid_search.best_params_)

    # 평균 평가 지표 계산
    mean_f1_score = np.mean(grid_search.cv_results_['mean_test_score'])
    print("Mean F1 Score:", mean_f1_score)

    return grid_search.best_params_, mean_f1_score

In [26]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_lgbm_with_best_params(X_train, y_train, X_test, y_test, best_params, k_fold=5):
    # Stratified k-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # LGBM 모델 초기화
    model = lgb.LGBMClassifier(**best_params)

    # 각 fold 별 평가 지표를 저장할 리스트 초기화
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # 모델 학습
        model.fit(X_train_fold, y_train_fold)

        # 테스트 데이터에 대한 예측 확률 얻기
        probabilities = model.predict_proba(X_test_fold)

        # threshold를 0.4로 설정하여 예측 클래스를 조정
        threshold = 0.5
        predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)

        # 평가 지표 계산
        accuracy = accuracy_score(y_test_fold, predicted_classes)
        precision = precision_score(y_test_fold, predicted_classes)
        recall = recall_score(y_test_fold, predicted_classes)
        f1 = f1_score(y_test_fold, predicted_classes)
        conf_matrix = confusion_matrix(y_test_fold, predicted_classes)

        # 각 fold 별 평가 지표를 리스트에 추가
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # 가장 좋은 f1-score 값을 가진 모델을 저장
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # 가장 좋은 f1-score 값을 가진 모델로 최종 예측 수행
    probabilities_final = best_model.predict_proba(X_test)
    y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)

    # 평가 지표 계산
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list

In [27]:
best_params, mean_f1_score = perform_lgbm_grid_search(X_train, y_train, k_fold=5)
evaluate_lgbm_with_best_params(X_train_int, y_train, X_test_int, y_test, best_params, k_fold=5)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best Hyperparameters: {'learning_rate': 50, 'max_depth': 3, 'n_estimators': 50, 'objective': 'binary', 'random_state': 0}
Mean F1 Score: 0.7544680793461281
Fold 1
Accuracy: 0.75
Precision: 0.6956521739130435
Recall: 0.8888888888888888
F1 score: 0.7804878048780488
Confusion Matrix:
[[11  7]
 [ 2 16]]
------------------------------
Fold 2
Accuracy: 0.5833333333333334
Precision: 0.6
Recall: 0.5
F1 score: 0.5454545454545454
Confusion Matrix:
[[12  6]
 [ 9  9]]
------------------------------
Fold 3
Accuracy: 0.8888888888888888
Precision: 0.8888888888888888
Recall: 0.8888888888888888
F1 score: 0.8888888888888888
Confusion Matrix:
[[16  2]
 [ 2 16]]
------------------------------
Fold 4
Accuracy: 0.8611111111111112
Precision: 0.8095238095238095
Recall: 0.9444444444444444
F1 score: 0.8717948717948718
Confusion Matrix:
[[14  4]
 [ 1 17]]
------------------------------
Fold 5
Accuracy: 0.6944444444444444
Precision: 0.7058823529411765


([0.75,
  0.5833333333333334,
  0.8888888888888888,
  0.8611111111111112,
  0.6944444444444444],
 [0.6956521739130435,
  0.6,
  0.8888888888888888,
  0.8095238095238095,
  0.7058823529411765],
 [0.8888888888888888,
  0.5,
  0.8888888888888888,
  0.9444444444444444,
  0.6666666666666666],
 [0.7804878048780488,
  0.5454545454545454,
  0.8888888888888888,
  0.8717948717948718,
  0.6857142857142857])

Final Test Results
Accuracy: 0.8055555555555556
Precision: 0.6875
Recall: 0.4074074074074074
F1 score: 0.5116279069767441
Confusion Matrix:
[[76  5]
 [16 11]]